In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load the data
customers = pd.read_csv('/content/Customers.csv')
products = pd.read_csv('/content/Products.csv')
transactions = pd.read_csv('/content/Transactions.csv')

# Display sample data for verification
customers.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [3]:
products.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [4]:
transactions.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [5]:
# Merge transactions with customers and products
transactions = transactions.merge(customers, on="CustomerID", how="left")
transactions = transactions.merge(products, on="ProductID", how="left")

# Check for missing values
print(transactions.isnull().sum())


TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price_x            0
CustomerName       0
Region             0
SignupDate         0
ProductName        0
Category           0
Price_y            0
dtype: int64


In [6]:
transactions

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics,459.86


In [6]:
# Convert categorical features to numerical using one-hot encoding
encoded_customers = pd.get_dummies(customers, columns=['Region'], drop_first=True)
encoded_products = pd.get_dummies(products, columns=['Category'], drop_first=True)

In [7]:
# Verify transformation
print(encoded_customers.head())

  CustomerID        CustomerName  SignupDate  Region_Europe  \
0      C0001    Lawrence Carroll  2022-07-10          False   
1      C0002      Elizabeth Lutz  2022-02-13          False   
2      C0003      Michael Rivera  2024-03-07          False   
3      C0004  Kathleen Rodriguez  2022-10-09          False   
4      C0005         Laura Weber  2022-08-15          False   

   Region_North America  Region_South America  
0                 False                  True  
1                 False                 False  
2                 False                  True  
3                 False                  True  
4                 False                 False  


In [8]:
print(encoded_products.head())

  ProductID              ProductName   Price  Category_Clothing  \
0      P001     ActiveWear Biography  169.30              False   
1      P002    ActiveWear Smartwatch  346.30              False   
2      P003  ComfortLiving Biography   44.12              False   
3      P004            BookWorld Rug   95.69              False   
4      P005          TechPro T-Shirt  429.31               True   

   Category_Electronics  Category_Home Decor  
0                 False                False  
1                  True                False  
2                 False                False  
3                 False                 True  
4                 False                False  


In [10]:
# Merge transactions with encoded customers and products before grouping
transactions = transactions.merge(encoded_customers, on="CustomerID", how="left")
transactions = transactions.merge(encoded_products, on="ProductID", how="left")
transactions

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName_x,Region,SignupDate_x,...,CustomerName_y,SignupDate_y,Region_Europe,Region_North America,Region_South America,ProductName_y,Price,Category_Clothing,Category_Electronics,Category_Home Decor
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,...,Andrea Jenkins,2022-12-03,True,False,False,ComfortLiving Bluetooth Speaker,300.68,False,True,False
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,...,Brittany Harvey,2024-09-04,False,False,False,ComfortLiving Bluetooth Speaker,300.68,False,True,False
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,...,Kathryn Stevens,2024-04-04,True,False,False,ComfortLiving Bluetooth Speaker,300.68,False,True,False
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,...,Travis Campbell,2024-04-11,False,False,True,ComfortLiving Bluetooth Speaker,300.68,False,True,False
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,...,Timothy Perez,2022-03-15,True,False,False,ComfortLiving Bluetooth Speaker,300.68,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,...,Jacob Holt,2022-01-22,False,False,True,SoundWave Smartwatch,459.86,False,True,False
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,...,Mrs. Kimberly Wright,2024-04-07,False,True,False,SoundWave Smartwatch,459.86,False,True,False
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,...,Tyler Haynes,2024-09-21,False,True,False,SoundWave Smartwatch,459.86,False,True,False
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,...,Joshua Hamilton,2024-11-11,False,False,False,SoundWave Smartwatch,459.86,False,True,False


In [11]:
# Aggregate transaction data to form customer profiles
customer_profiles = transactions.groupby('CustomerID').agg({
    'Price': 'mean',                      # Average price of products purchased
    'Quantity': 'sum',                    # Total quantity purchased
    'TotalValue': 'sum',                  # Total transaction value
    'Region_Europe': 'max',               # Region: Europe (one-hot feature)
    'Region_North America': 'max',        # Region: North America (one-hot feature)
    'Region_South America': 'max',        # Region: South America (one-hot feature)
    'Category_Clothing': 'sum',           # Total purchases in Clothing category
    'Category_Electronics': 'sum',        # Total purchases in Electronics category
    'Category_Home Decor': 'sum',         # Total purchases in Home Decor category
}).reset_index()

# Display the first few rows of the customer profiles
print(customer_profiles.head())


  CustomerID       Price  Quantity  TotalValue  Region_Europe  \
0      C0001  278.334000        12     3354.52          False   
1      C0002  208.920000        10     1862.74          False   
2      C0003  195.707500        14     2725.38          False   
3      C0004  240.636250        23     5354.88          False   
4      C0005  291.603333         7     2034.24          False   

   Region_North America  Region_South America  Category_Clothing  \
0                 False                  True                  0   
1                 False                 False                  2   
2                 False                  True                  1   
3                 False                  True                  0   
4                 False                 False                  0   

   Category_Electronics  Category_Home Decor  
0                     3                    1  
1                     0                    2  
2                     1                    2  
3           

In [12]:
# Select features for scaling (excluding CustomerID)
features_to_scale = customer_profiles.drop(columns=['CustomerID'])

# Apply StandardScaler to normalize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_to_scale)

# Create a DataFrame for scaled features
scaled_features_df = pd.DataFrame(
    scaled_features, columns=features_to_scale.columns
)

# Add the CustomerID column back to the scaled DataFrame
scaled_features_df['CustomerID'] = customer_profiles['CustomerID']

# Reorganize columns to place CustomerID first
customer_profiles_scaled = scaled_features_df[['CustomerID'] + list(features_to_scale.columns)]

# Display the scaled customer profiles
print(customer_profiles_scaled.head())


  CustomerID     Price  Quantity  TotalValue  Region_Europe  \
0      C0001  0.094670 -0.122033   -0.061701      -0.579284   
1      C0002 -0.904016 -0.448000   -0.877744      -0.579284   
2      C0003 -1.094109  0.203934   -0.405857      -0.579284   
3      C0004 -0.447702  1.670787    1.032547      -0.579284   
4      C0005  0.285581 -0.936951   -0.783929      -0.579284   

   Region_North America  Region_South America  Category_Clothing  \
0             -0.548319              1.540416          -1.041606   
1             -0.548319             -0.649175           0.776636   
2             -0.548319              1.540416          -0.132485   
3             -0.548319              1.540416          -1.041606   
4             -0.548319             -0.649175          -1.041606   

   Category_Electronics  Category_Home Decor  
0              1.550878            -0.221044  
1             -1.148463             0.676665  
2             -0.248683             0.676665  
3              0.651097 

In [13]:
# Exclude the 'CustomerID' column for similarity calculation
features_for_similarity = customer_profiles_scaled.drop(columns=['CustomerID'])

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(features_for_similarity)

# Convert the similarity matrix into a DataFrame for better interpretability
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=customer_profiles_scaled['CustomerID'],
    columns=customer_profiles_scaled['CustomerID']
)

# Display the top rows of the similarity DataFrame
print(similarity_df.head())


CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000 -0.525431  0.439679  0.521440  0.375248  0.250927   
C0002      -0.525431  1.000000  0.300977 -0.276977  0.042966 -0.074182   
C0003       0.439679  0.300977  1.000000  0.603203 -0.158692  0.393128   
C0004       0.521440 -0.276977  0.603203  1.000000 -0.250635  0.245930   
C0005       0.375248  0.042966 -0.158692 -0.250635  1.000000 -0.187985   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001       0.361006 -0.084329 -0.283385 -0.550088  ...  0.866519  0.731334   
C0002      -0.179985 -0.015276  0.197599  0.489740  ... -0.443740 -0.205136   
C0003      -0.358008  0.025561 -0.375410 -0.020660  ...  0.386117  0.375536   
C0004      -0.207901  0.395740 -0.817593 -0.534314  ...  0.239541 -0.044910   
C0005  

In [14]:
similarity_df

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,-0.525431,0.439679,0.521440,0.375248,0.250927,0.361006,-0.084329,-0.283385,-0.550088,...,0.866519,0.731334,-0.136689,-0.126372,0.192723,-0.399573,0.076497,-0.183354,0.051122,-0.377476
C0002,-0.525431,1.000000,0.300977,-0.276977,0.042966,-0.074182,-0.179985,-0.015276,0.197599,0.489740,...,-0.443740,-0.205136,0.250738,-0.112594,0.248145,-0.156922,-0.065697,0.209879,-0.019051,0.165060
C0003,0.439679,0.300977,1.000000,0.603203,-0.158692,0.393128,-0.358008,0.025561,-0.375410,-0.020660,...,0.386117,0.375536,-0.106783,-0.196152,0.788243,-0.495875,-0.164679,-0.235831,-0.125300,-0.308347
C0004,0.521440,-0.276977,0.603203,1.000000,-0.250635,0.245930,-0.207901,0.395740,-0.817593,-0.534314,...,0.239541,-0.044910,-0.336585,0.091636,0.707966,-0.141310,-0.247697,-0.711453,-0.098088,-0.041788
C0005,0.375248,0.042966,-0.158692,-0.250635,1.000000,-0.187985,0.909320,-0.277252,0.259279,-0.292752,...,0.382294,0.366797,0.458634,-0.064046,-0.503512,-0.146715,0.437700,0.375001,0.413222,-0.242685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C0196,-0.399573,-0.156922,-0.495875,-0.141310,-0.146715,0.221060,0.142727,-0.220618,0.285075,-0.025651,...,-0.366028,-0.416256,0.000299,-0.331989,-0.165681,1.000000,0.115498,0.131294,0.273089,0.191816
C0197,0.076497,-0.065697,-0.164679,-0.247697,0.437700,-0.515325,0.254575,-0.261201,0.586106,0.365868,...,0.053863,0.014576,0.133811,-0.208925,-0.485962,0.115498,1.000000,0.785962,0.932010,-0.551859
C0198,-0.183354,0.209879,-0.235831,-0.711453,0.375001,-0.268625,0.196563,-0.582176,0.922730,0.653591,...,0.034446,0.211188,0.327741,-0.413665,-0.578973,0.131294,0.785962,1.000000,0.635817,-0.403819


In [15]:
# Function to fetch the top N similar customers for a given customer ID
def get_top_n_similarities(customer_id, n=3):
    # Retrieve similarity scores for the given customer
    scores = similarity_df.loc[customer_id]
    # Exclude the customer itself and fetch the top N similar customers
    top_customers = scores.nlargest(n + 1).iloc[1:]
    return list(zip(top_customers.index, top_customers.values))

# Generate recommendations for the first 20 customers (C0001 - C0020)
lookalikes = {}
for customer_id in customers['CustomerID'][:20]:
    lookalikes[customer_id] = get_top_n_similarities(customer_id)

# Display the recommendations
for customer, recommendations in lookalikes.items():
    print(f"Customer {customer}: {recommendations}")


Customer C0001: [('C0120', 0.9181523234979794), ('C0191', 0.8665187771866838), ('C0118', 0.8357303970523413)]
Customer C0002: [('C0178', 0.8818140452547637), ('C0159', 0.8757706162002237), ('C0134', 0.8513878844697068)]
Customer C0003: [('C0031', 0.9075262861323825), ('C0147', 0.884141261715644), ('C0025', 0.8576413100243334)]
Customer C0004: [('C0113', 0.9533305623064582), ('C0163', 0.927491436843334), ('C0012', 0.9052755501670874)]
Customer C0005: [('C0146', 0.9376217194425887), ('C0186', 0.9190305130331514), ('C0007', 0.909319513330826)]
Customer C0006: [('C0187', 0.9020993861968956), ('C0171', 0.8601977704201105), ('C0126', 0.8107561723135949)]
Customer C0007: [('C0146', 0.9968253168585275), ('C0115', 0.9207042668548936), ('C0005', 0.909319513330826)]
Customer C0008: [('C0194', 0.8127485019915511), ('C0122', 0.7855072656722216), ('C0059', 0.7835327175870911)]
Customer C0009: [('C0061', 0.9230871202373055), ('C0198', 0.922729905572056), ('C0132', 0.8475049565087727)]
Customer C0010:

In [16]:
# Format recommendations as required
lookalikes_list = [
    {
        "CustomerID": cust,
        "Lookalikes": [
            {"SimilarCustomerID": sim[0], "Score": sim[1]} for sim in lookalikes[cust]
        ]
    }
    for cust in lookalikes
]

# Flatten the structure to meet the desired CSV format
output_data = []
for entry in lookalikes_list:
    customer_id = entry["CustomerID"]
    for similar in entry["Lookalikes"]:
        output_data.append({
            "CustomerID": customer_id,
            "SimilarCustomerID": similar["SimilarCustomerID"],
            "Score": similar["Score"]
        })

# Convert to DataFrame
lookalikes_df = pd.DataFrame(output_data)

# Save to CSV
lookalikes_df.to_csv("Lookalike.csv", index=False)

print("Lookalike.csv generated successfully.")


Lookalike.csv generated successfully.
